# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## Installing dependencies

In [1]:
%pip install git+https://github.com/mozilla-ai/structured-qa.git@5-add-benchmark

  Cloning https://github.com/mozilla-ai/structured-qa.git (to revision 5-add-benchmark) to /tmp/pip-req-build-q1o0cypa
  Running command git clone --filter=blob:none --quiet https://github.com/mozilla-ai/structured-qa.git /tmp/pip-req-build-q1o0cypa
  Running command git checkout -b 5-add-benchmark --track origin/5-add-benchmark
  Switched to a new branch '5-add-benchmark'
  Branch '5-add-benchmark' set up to track remote branch '5-add-benchmark' from 'origin'.
  Resolved https://github.com/mozilla-ai/structured-qa.git to commit a02ffd7c45a36261597af3f00a2316d7e349d05b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
!wget https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv

--2025-02-04 10:16:08--  https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21441 (21K) [text/plain]
Saving to: ‘structured_qa.csv’

structured_qa.csv   100%[===================>]  20.94K  --.-KB/s    in 0.001s  

2025-02-04 10:16:08 (13.8 MB/s) - ‘structured_qa.csv’ saved [21441/21441]



# Setup

In [3]:
import os
import google.generativeai as genai
from google.colab.userdata import get, SecretNotFoundError

try:
    genai.configure(api_key=get("GOOGLE_API_KEY"))
except SecretNotFoundError as e:
    raise RuntimeError("Please set the GOOGLE_API_KEY secret to your API key") from e
os.environ["LOGURU_LEVEL"] = "INFO"

In [4]:
from loguru import logger

## Function to Process a single Document

In [5]:
from structured_qa.config import FIND_PROMPT
from structured_qa.preprocessing import document_to_sections_dir
from structured_qa.workflow import find_retrieve_answer


ANSWER_WITH_TYPE_PROMPT = """
You are a rigorous assistant answering questions.
You must only answer based on the current information available which is:

```
{CURRENT_INFO}
```

If the current information available not enough to answer the question,
you must return "I need more info" srting and nothing else:

If the current information is enough to answer, you must return one of the following formats:
- YES/NO (for boolean questions)
- Number (for numeric questions)
- Single letter (for multiple-choice questions)
"""


def process_document(
    document_file,
    document_data,
    model,
    find_prompt: str = FIND_PROMPT,
    answer_prompt: str = ANSWER_WITH_TYPE_PROMPT,
):
    sections_dir = Path("sections") / Path(document_file).stem
    if not sections_dir.exists():
        logger.info("Splitting document into sections")
        document_to_sections_dir(document_file, sections_dir)

    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in document_data.iterrows():
        question = row["question"]
        logger.info(f"Question: {question}")
        answer, sections_checked = find_retrieve_answer(
            question, model, sections_dir, find_prompt, answer_prompt
        )
        logger.info(f"Answer: {answer}")
        answers[index] = answer
        sections[index] = sections_checked[-1] if sections_checked else None

    return answers, sections

## Load Model

In [6]:
from structured_qa.model_loaders import load_gemini_model

In [7]:
model = load_gemini_model("gemini-2.0-flash-exp", system_prompt=None)

# Run Benchmark

In [8]:
from pathlib import Path
from urllib.request import urlretrieve

import pandas as pd

logger.info("Loading input data")
data = pd.read_csv("structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)

for document_link, document_data in data.groupby("document"):
    logger.info(f"Downloading document {document_link}")
    downloaded_document = Path(f"{Path(document_link).name}.pdf")
    if not Path(downloaded_document).exists():
        urlretrieve(document_link, downloaded_document)
        logger.info(f"Downloaded {document_link} to {downloaded_document}")
    else:
        logger.info(f"File {downloaded_document} already exists")

    answers, sections = process_document(downloaded_document, document_data, model)

    for index in document_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-02-04 10:16:14.200 | INFO     | __main__:<cell line: 0>:6 - Loading input data
2025-02-04 10:16:14.223 | INFO     | __main__:<cell line: 0>:12 - Downloading document https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf
2025-02-04 10:16:14.616 | INFO     | __main__:<cell line: 0>:16 - Downloaded https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf to HAI_AI-Index-Report-2024.pdf.pdf
2025-02-04 10:16:14.623 | INFO     | __main__:process_document:34 - Splitting document into sections
2025-02-04 10:16:14.628 | INFO     | structured_qa.preprocessing:document_to_sections_dir:85 - Converting HAI_AI-Index-Report-2024.pdf.pdf


Processing HAI_AI-Index-Report-2024.pdf.pdf...
[                                        ] (0/502)[                                        ] (  1/502)[                                        ] (  2/502)[                                        ] (  3/502)[                                        ] (  4/502)[                                        ] (  5/502)[                                

2025-02-04 10:21:38.611 | SUCCESS  | structured_qa.preprocessing:document_to_sections_dir:88 - Converted
2025-02-04 10:21:38.612 | INFO     | structured_qa.preprocessing:document_to_sections_dir:90 - Extracting sections
2025-02-04 10:21:38.763 | SUCCESS  | structured_qa.preprocessing:document_to_sections_dir:94 - Found 84 sections
2025-02-04 10:21:38.764 | INFO     | structured_qa.preprocessing:document_to_sections_dir:95 - Writing sections to sections/HAI_AI-Index-Report-2024.pdf
2025-02-04 10:21:38.780 | SUCCESS  | structured_qa.preprocessing:document_to_sections_dir:103 - Done
2025-02-04 10:21:38.782 | INFO     | __main__:process_document:37 - Predicting
2025-02-04 10:21:38.785 | INFO     | __main__:process_document:42 - Question: which type of risk was identified as the leading concern globally? -A: Fairness risks. -B: Privacy and data governance risks. -C: Risks related to generative AI deployment.
2025-02-04 10:21:38.788 | INFO     | structured_qa.model_loaders:get_response:96 - 

]


2025-02-04 10:21:51.594 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 1
2025-02-04 10:21:52.767 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 2
2025-02-04 10:21:53.990 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 3
2025-02-04 10:21:55.165 | INFO     | __main__:process_document:46 - Answer: B

2025-02-04 10:21:55.168 | INFO     | __main__:process_document:42 - Question: In which geographical area were fairness risks selected by the smallest percentage of respondents? -A: Asia. -B: Europe. -C: North America.
2025-02-04 10:21:55.171 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 4
2025-02-04 10:21:56.295 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 5
2025-02-04 10:21:57.366 | INFO     | __main__:process_document:46 - Answer: C

2025-02-04 10:21:57.370 | INFO     | __main__:process_document:42 - Question: What is a major consequence of the rising trai

...

2025-02-04 11:27:17.430 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 1
2025-02-04 11:27:18.424 | INFO     | __main__:process_document:46 - Answer: 3

2025-02-04 11:27:18.426 | INFO     | __main__:process_document:42 - Question: How many victory conditions are there?
2025-02-04 11:27:18.427 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 2
2025-02-04 11:27:19.420 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 3
2025-02-04 11:27:20.367 | INFO     | __main__:process_document:46 - Answer: 3

2025-02-04 11:27:20.369 | INFO     | __main__:process_document:42 - Question: How many different races are there?
2025-02-04 11:27:20.371 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 4
2025-02-04 11:27:21.342 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 5
2025-02-04 11:27:22.412 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 6
2025-02-04 

Processing is_eotn_rulebook.pdf.pdf...
[                                        ] (0/12)===[===                                     ] ( 1/12)===[======                                  ] ( 2/12)====[==========                              ] ( 3/12)===[=============                           ] ( 4/12)===[================                        ] ( 5/12)====[====================                    ] ( 6/12)===

2025-02-04 11:51:02.798 | SUCCESS  | structured_qa.preprocessing:document_to_sections_dir:88 - Converted
2025-02-04 11:51:02.799 | INFO     | structured_qa.preprocessing:document_to_sections_dir:90 - Extracting sections
2025-02-04 11:51:02.808 | SUCCESS  | structured_qa.preprocessing:document_to_sections_dir:94 - Found 40 sections
2025-02-04 11:51:02.811 | INFO     | structured_qa.preprocessing:document_to_sections_dir:95 - Writing sections to sections/is_eotn_rulebook.pdf
2025-02-04 11:51:02.819 | SUCCESS  | structured_qa.preprocessing:document_to_sections_dir:103 - Done
2025-02-04 11:51:02.821 | INFO     | __main__:process_document:37 - Predicting
2025-02-04 11:51:02.825 | INFO     | __main__:process_document:42 - Question: What is the maximum number of cards a player may acquire during the lookout phase?
2025-02-04 11:51:02.828 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 9
2025-02-04 11:51:02.830 | INFO     | structured_qa.model_loaders:get_response:99 

]


2025-02-04 11:52:04.181 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 1
2025-02-04 11:52:05.176 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 2
2025-02-04 11:52:06.424 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 3
2025-02-04 11:52:07.419 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 4
2025-02-04 11:52:08.666 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 5
2025-02-04 11:52:09.661 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 6
2025-02-04 11:52:10.705 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 7
2025-02-04 11:52:11.877 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 8
2025-02-04 11:52:13.150 | INFO     | structured_qa.model_loaders:get_response:96 - Current calls: 9
2025-02-04 11:52:13.152 | INFO     | structured_qa.model_loaders:get_response:99 - Waiting for 60 se

In [9]:
results = pd.read_csv("results.csv")
for index, result in results.iterrows():
    results.loc[index, "pred_answer"] = result["pred_answer"].strip()
    if result["pred_answer"].startswith(
        (f"-{result['answer']}", f"{result['answer']}")
    ):
        results.loc[index, "pred_answer"] = result["answer"]
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,section,question,answer,pred_answer,pred_section
12,12,https://arxiv.org/pdf/2210.05189,3 Experimental Results,How many layers are in the toy model (y = x^2)?,3,NOT FOUND,Caglar Aytekin AI Lead AAC Technologies
28,28,https://arxiv.org/pdf/2201.11903,3.1 Experimental Setup,How many large language models were evaluated?,5,3,Abstract
32,32,https://arxiv.org/pdf/2201.11903,5 Symbolic Reasoning,Which symbolic reasoning task is used as an ou...,A,YES,5 Symbolic Reasoning
33,33,https://arxiv.org/pdf/2201.11903,3.4 Robustness of Chain of Thought,How many annotators provided independent chain...,3,2,H Appendix: Alternate Annotators for MWP
34,34,https://arxiv.org/pdf/2201.11903,3.2 Results,How many random samples were examined to under...,100,NOT FOUND,Chain-of-Thought Prompting Elicits Reasoning i...
45,45,https://github.com/mozilla-ai/structured-qa/re...,CARD AND TILE EFFECTS,Which type of cards provide coins? -A: Gray -B...,B,NOT FOUND,4
48,48,https://github.com/mozilla-ai/structured-qa/re...,END OF THE GAME,Can the game end in a tie?,YES,NO,OVERVIEW AND GOAL
50,50,https://github.com/mozilla-ai/structured-qa/re...,LOOKOUT PHASE,What is the maximum number of cards a player m...,4,NOT FOUND,1 3 3 5 7 8
51,51,https://github.com/mozilla-ai/structured-qa/re...,LOOKOUT PHASE,Is there a limit to the number of cards a play...,NO,YES,CLEANUP PHASE players with extra Goods if enou...
54,54,https://github.com/mozilla-ai/structured-qa/re...,EXPEDITION PHASE,Can players conquer and pillage the same islan...,NO,YES,EXPEDITION PHASE


In [10]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.8640776699029126